In [1]:
import pandas as pd
import os
import detect_simpsons_paradox as dsp
import numpy as np
import itertools

In [2]:
labeled_df = dsp.labeledDataFrame('../data/ldf_state_hit_rate_min_cols_COCTFLILMDMAMOMTNENCOHRISCTXVTWAWI')
labeled_df.meta_df

,dtype,var_type,role,isCount,weighting_var
variable,,,,,
state,object,categorical,"[explanatory, groupby]",False,NaN
year,int64,ordinal,[trend],False,NaN
driver_gender,object,binary,"[explanatory, groupby]",False,NaN
driver_race,object,categorical,"[explanatory, groupby]",False,NaN
decriminalization,int64,binary,"[explanatory, groupby]",False,NaN
medical,int64,binary,"[explanatory, groupby]",False,NaN
recreational,int64,binary,"[explanatory, groupby]",False,NaN
no_reforms,int64,binary,"[explanatory, groupby]",False,NaN
search_conducted_false,float64,continuous,[trend],True,NaN


In [3]:
rankobj = dsp.mean_rank_trend()
linregobj = dsp.linear_trend()

The tren objects above will compute all pairs of given types, but what if we want to define custom trends?  We can do that by overloading existing types.  We'll overload only the get_trend_vars() function for now, but the other functions can also be overloaded or a totally new trend can be added as long as it is compatible.

In [4]:
class min_lin_reg(dsp.linear_trend):
    symmetric_vars = False # tell it not to do combinations
    
    def get_trend_vars(self,labeled_df):
        """
        """
        x_vars = ['year']
        y_vars = ['search_conducted_rate','contraband_found_rate','hit_rate','num_stops','search_conducted_true']
        
        self.regression_vars = list(itertools.product(x_vars,y_vars))
        return self.regression_vars
    
min_lin_reg_obj = min_lin_reg()

In [5]:
labeled_df.get_subgroup_trends_1lev([min_lin_reg_obj,rankobj])
labeled_df.result_df.sample(10)

,feat1,feat2,subgroup_trend,subgroup,subgroup_trendquality,group_feat,trend_type,agg_trend,agg_trendquality
27,year,search_conducted_rate,-0.001562,1,-0.082898,no_reforms,lin_reg,-0.000353,-0.039030
31,year,contraband_found_rate,NaN,IL,NaN,state,lin_reg,NaN,NaN
79,year,hit_rate,NaN,1,NaN,medical,lin_reg,NaN,NaN
0,year,search_conducted_rate,-0.010591,AZ,-0.247018,state,lin_reg,-0.000353,-0.039030
137,year,search_conducted_true,-306.219101,1,-0.159283,recreational,lin_reg,-55.628632,-0.062242
54,year,contraband_found_rate,NaN,0,NaN,no_reforms,lin_reg,NaN,NaN
80,year,hit_rate,NaN,0,NaN,recreational,lin_reg,NaN,NaN
9,year,search_conducted_rate,-0.001498,TX,-0.361134,state,lin_reg,-0.000353,-0.039030
72,year,hit_rate,NaN,Black,NaN,driver_race,lin_reg,NaN,NaN
77,year,hit_rate,NaN,1,NaN,decriminalization,lin_reg,NaN,NaN
